# Importing the data and selecting subset
The data is first imported from the SQLite file and converted into a pandas dataframe. After that, all N/A and duplicate records are dropped, and a the dataset is subsetted randomly. 

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect("./data/properties_cleaned.db")
df = pd.read_sql_query(f"SELECT * FROM properties", conn)
conn.close()

#['bedrooms', 'building_form', 'city', 'commercial_space',
# 'days_on_market', 'energy_efficient', 'energy_label', 'fixer_upper',
# 'has_balcony', 'has_garden', 'has_heat_pump', 'has_roof_terrace',
# 'has_solar_panels', 'latitude', 'living_area', 'longitude',
# 'national_monument', 'object_type', 'plot_area_m2', 'postcode4',
# 'price', 'province', 'publication_date', 'rooms', 'url', 'year_built',
# 'has_basement', 'has_attic', 'stories'],

df = df.dropna()
df = df.drop_duplicates()
df = df.sample(n=200000, random_state=1)

# Select subset
df = df[['price', 'bedrooms', 'building_form', 
         #'city', 
         'commercial_space',
         'days_on_market', 
         'energy_efficient', 'energy_label', 'fixer_upper',
         'has_balcony', 'has_garden', 'has_heat_pump', 'has_roof_terrace',
         'has_solar_panels', 'latitude', 'living_area', 'longitude',
         'national_monument', 
         'object_type', 'plot_area_m2', #'postcode4',
         'province', 
         #'publication_date', 
         'rooms', 'year_built',
         'has_basement', 'has_attic', 'stories']]

# Preprocessing and analyzing the data prior to modeling
Before the model can be created, the data has to be preprocessed and analyzed for things such as multicollinearity.

## Converting datatypes
All columns have to be converted to the proper datatypes to allow modeling

In [11]:
# CATEGORY
df['building_form'] = df['building_form'].astype('category')
df['object_type'] = df['object_type'].astype('category')
df['province'] = df['province'].astype('category')

# Ordinal category for energy_label
df['energy_label'] = pd.Categorical(
    df['energy_label'],
    categories=['g', 'f', 'e', 'd', 'c', 'b', 'a'],
    ordered=True
)

# BOOL
df['has_balcony'] = df['has_balcony'].astype(bool)
df['has_garden'] = df['has_garden'].astype(bool)
df['has_heat_pump'] = df['has_heat_pump'].astype(bool)
df['has_roof_terrace'] = df['has_roof_terrace'].astype(bool)
df['has_solar_panels'] = df['has_solar_panels'].astype(bool)
df['has_basement'] = df['has_basement'].astype(bool)
df['has_attic'] = df['has_attic'].astype(bool)
df['fixer_upper'] = df['fixer_upper'].astype(bool)
df['energy_efficient'] = df['energy_efficient'].astype(bool)
df['commercial_space'] = df['commercial_space'].astype(bool)
df['national_monument'] = df['national_monument'].astype(bool)

# INT
df['rooms'] = df['rooms'].astype(int)
df['year_built'] = df['year_built'].astype(int)
df['living_area'] = df['living_area'].astype(int)
df['plot_area_m2'] = df['plot_area_m2'].astype(int)
df['days_on_market'] = df['days_on_market'].astype(int)
df['bedrooms'] = df['bedrooms'].astype(int)
df['stories'] = df['stories'].astype(int)

# FLOAT
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['price'] = df['price'].str.replace('€', '').str.replace('.', '').astype(float)

# print data types
print(df.dtypes)

price                 float64
bedrooms                int64
building_form        category
commercial_space         bool
days_on_market          int64
energy_efficient         bool
energy_label         category
fixer_upper              bool
has_balcony              bool
has_garden               bool
has_heat_pump            bool
has_roof_terrace         bool
has_solar_panels         bool
latitude              float64
living_area             int64
longitude             float64
national_monument        bool
object_type          category
plot_area_m2            int64
province             category
rooms                   int64
year_built              int64
has_basement             bool
has_attic                bool
stories                 int64
dtype: object


In [12]:
print(len(df))

def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Remove outliers for specified columns
for col in ['bedrooms', 'living_area', 'plot_area_m2', 'rooms', 'year_built', 'stories', 'price']:
    df = remove_outliers_iqr(df, col)
print(len(df))

100000
83748


## Check for Multicolinearity
Multicollinearity doesnt impact rf, but important for interpretting 

In [13]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

numeric_vars = df.select_dtypes(include=['number']).drop(columns='price')
X = add_constant(numeric_vars)  # A constant is added
 
# Compute VIF for each numeric feature
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values(by='VIF', ascending=False)  # Sort by VIF values

print(vif_data)

          feature           VIF
0           const  17227.222787
7           rooms      5.619852
1        bedrooms      5.506775
4     living_area      2.852388
6    plot_area_m2      1.799781
9         stories      1.553476
5       longitude      1.239470
3        latitude      1.120167
8      year_built      1.118899
2  days_on_market      1.003113


The VIF values show signs of multicolliniearity for bedrooms and rooms. Therefore, the rooms variable will be converted to a non_bedroom_rooms variable. This will give the variable a different meaning, while still maintaining context.

In [14]:
if 'rooms' in df.columns:
    df['non_bedroom_rooms'] = df['rooms'] - df['bedrooms'].astype(int)
    df = df.drop(columns=['rooms'])

# Neural Network
Create a simple neural network model

## Initial Model

In [15]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.model_selection import train_test_split

y = df['price']
X = pd.get_dummies(df.drop(columns=['price']), drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#nn = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
nn = MLPRegressor(
    hidden_layer_sizes=(50, 25),
    activation='relu',
    solver='lbfgs',
    alpha=0.001,
    learning_rate='adaptive',
    max_iter=1000,
    early_stopping=True,
    random_state=42
)


nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Neural Network MAE: {mae:.2f}")
print(f"Neural Network MSE: {mse:.2f}")
print(f"Neural Network R2: {r2:.2f}")



Neural Network MAE: 70221.34
Neural Network MSE: 8321957116.72
Neural Network R2: 0.56


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor

# Define the parameter grid to search
param_dist = {
    'hidden_layer_sizes': [(64, 32), (100, 50), (128, 64), (100,), (50, 25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [300, 500, 1000]
}

# Use a smaller subset for speed if needed
X_sample = X_train#.sample(n=2000, random_state=42)
y_sample = y_train.loc[X_sample.index]

# Set up the randomized search
random_search = RandomizedSearchCV(
    MLPRegressor(early_stopping=True, random_state=42),
    param_distributions=param_dist,
    n_iter=15,  # Number of random combinations to try
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_sample, y_sample)

print("Best parameters found:", random_search.best_params_)
print("Best MAE:", -random_search.best_score_)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) 

KeyboardInterrupt: 

In [12]:
# Run prediction on one property

property_example = {
    'bedrooms': 3,
    'building_form': 'detached',
    'commercial_space': False,
    'days_on_market': 30,
    'energy_efficient': True,
    'energy_label': 'c',
    'fixer_upper': False,
    'has_balcony': True,
    'has_garden': True,
    'has_heat_pump': False,
    'has_roof_terrace': False,
    'has_solar_panels': True,
    'latitude': 52.3676,
    'living_area': 120,
    'longitude': 4.9041,
    'national_monument': False,
    'object_type': 'house',
    'plot_area_m2': 200,
    # 'postcode4': '',  # Not used
    'province': 'North Holland',
    # 'publication_date': '',  # Not used
    'rooms': 5,  # Total rooms including bedrooms
    'year_built': 1990,
    'has_basement': False,
    'has_attic': True,
    'stories': 2
}

property_df = pd.DataFrame([property_example])
property_df = pd.get_dummies(property_df, drop_first=True)

# Ensure the same columns as in the training set
missing_cols = set(X.columns) - set(property_df.columns)
for col in missing_cols:
    property_df[col] = 0
property_df = property_df[X.columns]  # Reorder columns to match training set
predicted_price = rf.predict(property_df)
print("Predicted price for the example property: €{:.2f}".format(predicted_price[0]))


NameError: name 'rf' is not defined

## Feature Importance

### SHAP

In [ ]:
import shap
import numpy as np

subset_idx = np.random.choice(X.index, size=1000, replace=False)
X_subset = X.loc[subset_idx]
y_subset = y.loc[subset_idx]

# Fit the RandomForestRegressor on the subset
rf.fit(X_subset, y_subset)

# Create a SHAP explainer for the trained model
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_subset)

# Plot SHAP summary for feature importance
shap.summary_plot(shap_values, X_subset, plot_type="bar")

NameError: name 'rf' is not defined